# Hot start analysis

Notebook to analyse difference between hot start and the final solution.

Import required functions and import, or declare, constants:

In [1]:
from modules.helper_functions_tsp import (hot_start, 
                                         find_problem_size,
                                         find_distances_array,
                                         hot_start_list_to_string,
                                         cost_fn_fact,
                                         convert_integer_to_binary_list,
                                         convert_bit_string_to_cycle,
                                         )   

import pandas as pd
from pathlib import Path
import numpy as np
from typing import Callable

from modules.config import (DECODING_FORMULATION, 
                            GRAY, 
                            RESULTS_DIR
                            )

BRUTE_FORCE_START_LOCATION = 4  #only used for brute force
BRUTE_FORCE_END_LOCATION = 12  #only used for brute force
BRUTE_FORCE = False

HOT_START_QUALITY = True
HOT_START_QUALITY_LIST = [4, 5, 6, 7, 8, 9, 10, 11, 12, 15, 17, 26, 42, 48]

Declare dataframe:

In [ ]:
print(f'Running experiment with {DECODING_FORMULATION} decoding formulation and gray code = {GRAY}.')
if BRUTE_FORCE:
    print('Brute force is enabled.')
    df = pd.DataFrame(columns=['locations', 
                            'qubits', 
                            'best dist',
                            'hot start dist',
                            'quality',
                            'lowest hamming dist',])
if HOT_START_QUALITY:
    print('Hot start quality is enabled.')
    df2 = pd.DataFrame(columns=['locations', 
                            'qubits', 
                            'best dist',
                            'hot start dist',
                            'quality',
                            ]
        )


Running experiment with original decoding formulation and False gray code.
Hot start quality is enabled.


Define required function modules:

In [3]:
def find_hot_start_distance(distance_array: np.array, 
                            locations:int, 
                            cost_fn: Callable
                            ) -> tuple:
    """find the hot start distance and the corresponding binary list"""
    hot_start_list = hot_start(distance_array, locations)
    print(f'The hot start location list is {hot_start_list}')
    bin_hot_start_list =  hot_start_list_to_string(hot_start_list, 
                                                locations, 
                                                GRAY, 
                                                DECODING_FORMULATION
                                                )
    print(f'This is equivalent to a binary list: {bin_hot_start_list}')

    hot_start_distance = cost_fn(bin_hot_start_list)
    return hot_start_distance, bin_hot_start_list

In [4]:
def find_best_distance(locations: int, 
                       qubits :int, 
                       cost_fn: Callable,
                       )-> tuple:
    """find the best distance from brute force by iterating through all possible routes"""
    lowest_to_date = locations*2 * 100 #to high
    lowest_list, route_list = [] , []
    for integer in range(2**qubits):
        binary_string_list = convert_integer_to_binary_list(integer, length=qubits, gray=GRAY)
        cost = cost_fn(binary_string_list)
        if cost < lowest_to_date:
            lowest_list, route_list = [] , []
            lowest_to_date = cost
            lowest_list.append(binary_string_list)
            route = convert_bit_string_to_cycle(binary_string_list, locations)
            route_list.append(route)
        elif cost == lowest_to_date:
            lowest_list.append(binary_string_list)
            route = convert_bit_string_to_cycle(binary_string_list, locations)
            route_list.append(route)
    return lowest_to_date, lowest_list, route_list

In [5]:
def find_hamming_distance(a:list, b:list) -> int:
    """find hamming distance between two binary lists"""
    return sum(bit1 != bit2 for bit1, bit2 in zip(a, b))

In [6]:
def find_lowest_hamming_distance(qubits: int, 
                                 lowest_list: list, 
                                 bin_hot_start_list: list,
                                 )-> int:
    """find the hamming distance between the hot start and the brute force solution"""
    lowest_hamming_distance = qubits #too high
    for routes in lowest_list:
        hamming_distance = find_hamming_distance(routes, bin_hot_start_list)
        if hamming_distance < lowest_hamming_distance:
            lowest_hamming_distance = hamming_distance
    return lowest_hamming_distance

In [7]:
def save_file(filename: str, 
               df: pd.DataFrame, 
               ) -> None:
    filepath = Path(RESULTS_DIR).joinpath(filename)
    try:
        df.to_csv(filepath, index=False)
        print(f'CSV file saved successfully at {filepath}')
    except Exception as e:
        print(f'An error occurred while saving the CSV: {e}')

Loop over the range of iterations and find the hot start and brute force distances and bit strings.  Find the minimum hot start distance between them:

In [8]:
if BRUTE_FORCE:
    for locations in range(BRUTE_FORCE_START_LOCATION, BRUTE_FORCE_END_LOCATION+1):
        print(f'Processing {locations} locations.')
        qubits = find_problem_size(locations, DECODING_FORMULATION)
        distance_array, best_dist = find_distances_array(locations, print_comments=True)
        cost_fn = cost_fn_fact(locations,
                            distance_array, 
                            GRAY, 
                            method = DECODING_FORMULATION,
                            )
        ##find the hot start distance and the corresponding binary list
        hot_start_distance, bin_hot_start_list = find_hot_start_distance(distance_array, locations, cost_fn)
        quality = round(best_dist / hot_start_distance, 3)
        print(f'The hot start distance is {hot_start_distance}, compared to a best distance of {best_dist}.')
        print(f'The solution quality of the hot start is {quality:.3f}')

        #find the best distance from brute force by iterating through all possible routes
        lowest_to_date, lowest_list, route_list = find_best_distance(locations, qubits, cost_fn)
        print(f'The lowest distance found by a brute force search is {lowest_to_date}')
        print(f'The list of binary strings is {lowest_list}')
        print(f'The corresponding routes are {route_list}')

        ##find the hamming distance between the hot start and the brute force solution
        lowest_hamming_distance = find_lowest_hamming_distance(qubits, lowest_list, bin_hot_start_list)
        print(f'The lowest hamming distance is {lowest_hamming_distance}')

        new_row = {'locations': locations,  
                'qubits': qubits, 
                'best dist': best_dist,
                'hot start dist': hot_start_distance,
                'quality': quality,
                'lowest hamming dist': lowest_hamming_distance,
                }
        # Append the new row to the DataFrame
        df.loc[len(df)] = new_row


print results:

In [9]:
if BRUTE_FORCE:
    filename = 'hot_start_hamming_distance.csv'
    print(df.to_string(index=False))
    save_file(filename, df)
    

In [10]:
if HOT_START_QUALITY:
    for locations in HOT_START_QUALITY_LIST:
        print(f'Processing {locations} locations.')
        qubits = find_problem_size(locations, DECODING_FORMULATION)
        distance_array, best_dist = find_distances_array(locations, print_comments=True)
        cost_fn = cost_fn_fact(locations,
                            distance_array, 
                            GRAY, 
                            method = DECODING_FORMULATION,
                            )
        ##find the hot start distance and the corresponding binary list
        hot_start_distance, bin_hot_start_list = find_hot_start_distance(distance_array, locations, cost_fn)
        quality = round(best_dist / hot_start_distance, 3)
        print(f'The hot start distance is {hot_start_distance}, compared to a best distance of {best_dist}.')
        print(f'The solution quality of the hot start is {quality:.3f}')
        
        new_row = {
            'locations': locations,  
            'qubits': qubits, 
            'best dist': best_dist,
            'hot start dist': hot_start_distance,
            'quality': quality,
            }
        df2.loc[len(df2)] = new_row


Processing 4 locations.
Reading distance data
Data will be read from filename networks\four_d.txt.
It is known that the shortest distance is 21
The hot start location list is [0, 1, 2, 3]
This is equivalent to a binary list: [0, 0, 0]
The hot start distance is 21.0, compared to a best distance of 21.
The solution quality of the hot start is 1.000
Processing 5 locations.
Reading distance data
Data will be read from filename networks\five_d.txt.
It is known that the shortest distance is 19
The hot start location list is [0, 3, 2, 1, 4]
This is equivalent to a binary list: [1, 0, 0, 1, 0]
The hot start distance is 21.0, compared to a best distance of 19.
The solution quality of the hot start is 0.905
Processing 6 locations.
Reading distance data
Data will be read from filename networks\sim_dist_6_locs.txt.
It is known that the shortest distance is 241.0
The hot start location list is [0, 3, 5, 4, 1, 2]
This is equivalent to a binary list: [0, 1, 0, 1, 1, 1, 0, 0]
The hot start distance is

In [11]:
if HOT_START_QUALITY:
    print(df2.to_string(index=False))
    filename = 'hot_start_quality.csv'
    save_file(filename, df2)

 locations  qubits  best dist  hot start dist  quality
         4       3       21.0            21.0    1.000
         5       5       19.0            21.0    0.905
         6       8      241.0           279.6    0.862
         7      11      276.2           314.8    0.877
         8      14      277.2           315.8    0.878
         9      17      286.7           339.8    0.844
        10      21      290.2           312.3    0.929
        11      25      253.0           299.0    0.846
        12      29      297.2           385.5    0.771
        15      41      291.0           291.0    1.000
        17      49     2085.0          2187.0    0.953
        26      94      937.0          1112.0    0.843
        42     183      699.0           956.0    0.731
        48     219    10628.0         40551.0    0.262
CSV file saved successfully at results\hot_start_quality.csv
